In [34]:
import pandas as pd
import numpy as np
import anndata as ad

In [35]:
df = pd.DataFrame(np.random.rand(5, 5))
df.index = ['cell1', 'cell2', 'cell3', 'cell4', 'cell5']
df.columns = ['gene1', 'gene2', 'gene3', 'gene4', 'gene5']
adata = ad.AnnData(X=df)

In [36]:
adata[:5, ['gene1', 'gene2']] # view
adata_subset = adata[:5, ['gene1', 'gene2']] # view
adata_subset_copy = adata[:5, ['gene1', 'gene2']].copy() # copy


In [37]:
adata_subset_copy[:2, 'gene1'] = 0 # change inplace
adata_subset_copy.X

array([[0.        , 0.46490831],
       [0.        , 0.13772762],
       [0.78747455, 0.87833713],
       [0.85835941, 0.91856311],
       [0.81444342, 0.50609847]])

In [38]:
adata_subset[:2, 'gene1'] = 0 # change inplace, fails because [] assignment does not work on view

/var/folders/2l/hhd_z4hx3070zw8rlj4c3l940000gn/T/ipykernel_2057/3461208026.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_subset.obs['gene1'] = 0 # change inplace


In [31]:
adata_subset.obs['status'] = 'test' # works, changes adata_subset to its own object
# same for any change of obs/var

# now, 
adata_subset[:2, 'gene1'] = 0 # works, changes adata_subset to its own object

/var/folders/2l/hhd_z4hx3070zw8rlj4c3l940000gn/T/ipykernel_2057/3769873534.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_subset.obs['status'] = 'test' # works, changes adata_subset to its own object


In [33]:
adata_subset.X

array([[0.        , 0.62888733],
       [0.        , 0.13615301],
       [0.63973993, 0.87448081],
       [0.20525594, 0.73308098],
       [0.85619766, 0.73160866]])

In [41]:
adata.layers['test'] = adata.X[:2, :2] # works

ValueError: Value passed for key 'test' is of incorrect shape. Values of layers must match dimensions (0, 1) of parent. Value had shape (2, 2) while it should have had (5, 5).